# Missions Deck Adjuster

This notebook reads `missions.md`, scores missions by the numeric score in each line, and moves the top 20 scores to the `Highways` section. The rest go to `Standard`.

Assumptions:
- `missions.md` has sections `## Standard` and `## Highways`
- Each mission line ends with `(score: N)`


In [1]:
from pathlib import Path
import re

md_path = Path('missions.md')
text = md_path.read_text()

# Split by sections
sections = re.split(r'^##\s+', text, flags=re.MULTILINE)
if len(sections) < 2:
    raise ValueError('Expected sections with ## Standard and ## Highways')

# sections format: [before, 'Standard
# ...','Highways
# ...'] (order may vary)
parsed = {}
for s in sections[1:]:
    header, *body = s.splitlines()
    header = header.strip()
    parsed[header] = '\n'.join(body).strip()

def parse_missions(block):
    missions = []
    for line in block.splitlines():
        line = line.strip()
        if not line or not line.startswith('- '):
            continue
        m = re.search(r'\(score:\s*(\d+)\)', line)
        if not m:
            continue
        score = int(m.group(1))
        missions.append((score, line))
    return missions

standard = parse_missions(parsed.get('Standard', ''))
highways = parse_missions(parsed.get('Highways', ''))
all_missions = standard + highways

if not all_missions:
    raise ValueError('No missions found to adjust.')

# sort by score descending, stable by line
all_missions.sort(key=lambda x: (-x[0], x[1]))
top_n = 20
highways_new = all_missions[:top_n]
standard_new = all_missions[top_n:]

def render_section(title, missions):
    lines = [f'## {title}']
    if not missions:
        lines.append('(none)')
        return lines
    for score, line in missions:
        lines.append(line)
    lines.append('')
    return lines

# Preserve header above first section
header = sections[0].rstrip()
out_lines = []
if header:
    out_lines.append(header)
    out_lines.append('')
out_lines += render_section('Standard', standard_new)
out_lines += render_section('Highways', highways_new)

md_path.write_text('\n'.join(out_lines).rstrip() + '\n')
print(f'Updated {md_path} with {len(highways_new)} highways and {len(standard_new)} standard missions.')


Updated missions.md with 20 highways and 80 standard missions.


In [3]:
# Profiling: route and mission stats (color + land/ocean portion)
from collections import defaultdict
import heapq
import math
import re

routes_path = Path('routes.md')
missions_path = Path('missions.md')


def parse_routes(text):
    routes = []
    for line in text.splitlines():
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        tokens = line.split()
        if len(tokens) < 4:
            raise ValueError(f"Invalid route line: {line}")
        length_token, color_token, type_token = tokens[-3:]
        pair = " ".join(tokens[:-3]).strip()
        if '-' not in pair:
            raise ValueError(f"Invalid city pair (missing '-'): {pair}")
        a, b = [x.strip() for x in pair.split('-', 1)]
        length = float(length_token)
        color = color_token.lower()
        if color == 'gray':
            color = 'grey'
        rtype = type_token.lower()
        if rtype not in {'land', 'ocean'}:
            raise ValueError(f"Invalid route type: {rtype}")
        routes.append({
            'a': a,
            'b': b,
            'length': length,
            'color': color,
            'type': rtype,
        })
    return routes


def summarize_edges(edges, label):
    total_len = sum(e['length'] for e in edges)
    total_edges = len(edges)
    land_len = sum(e['length'] for e in edges if e['type'] == 'land')
    ocean_len = sum(e['length'] for e in edges if e['type'] == 'ocean')
    land_edges = sum(1 for e in edges if e['type'] == 'land')
    ocean_edges = sum(1 for e in edges if e['type'] == 'ocean')
    color_totals = defaultdict(float)
    for e in edges:
        color_totals[e['color']] += e['length']

    print(label)
    print(f"  edges: {total_edges}, total length: {total_len:.2f}")
    if total_edges:
        print(f"  land edges: {land_edges} ({land_edges/total_edges:.1%})")
        print(f"  ocean edges: {ocean_edges} ({ocean_edges/total_edges:.1%})")
    if total_len > 0:
        print(f"  land length: {land_len:.2f} ({land_len/total_len:.1%})")
        print(f"  ocean length: {ocean_len:.2f} ({ocean_len/total_len:.1%})")
    print("  color totals (by length):")
    for color in sorted(color_totals):
        if total_len > 0:
            pct = color_totals[color] / total_len
            print(f"    {color}: {color_totals[color]:.2f} ({pct:.1%})")
        else:
            print(f"    {color}: {color_totals[color]:.2f}")
    print()


routes = parse_routes(routes_path.read_text())

summarize_edges(routes, "All routes stats")


# Missions -> shortest path stats
mission_line_re = re.compile(r'^-\s*(.*?)\s*(?:—|-)\s*(.*?)\s*\(score:\s*\d+\)', re.UNICODE)


def split_mission_sections(text):
    sections = re.split(r'^##\s+', text, flags=re.MULTILINE)
    parsed = {}
    for s in sections[1:]:
        header, *body = s.splitlines()
        parsed[header.strip()] = "\n".join(body).strip()
    return parsed


def parse_missions(block):
    missions = []
    for line in block.splitlines():
        line = line.strip()
        if not line:
            continue
        m = mission_line_re.match(line)
        if not m:
            continue
        a = m.group(1).strip()
        b = m.group(2).strip()
        missions.append((a, b))
    return missions


adj = defaultdict(list)
for e in routes:
    adj[e['a']].append((e['b'], e['length'], e['color'], e['type']))
    adj[e['b']].append((e['a'], e['length'], e['color'], e['type']))


def shortest_path_edges(start, end):
    if start == end:
        return []
    pq = [(0.0, 0, start)]  # cost, hops, node
    best = {start: (0.0, 0)}
    prev = {}
    while pq:
        cost, hops, node = heapq.heappop(pq)
        if (cost, hops) != best.get(node, (math.inf, math.inf)):
            continue
        if node == end:
            break
        for nxt, length, color, rtype in adj.get(node, []):
            ncost = cost + length
            nhops = hops + 1
            cur_best = best.get(nxt, (math.inf, math.inf))
            if (ncost, nhops) < cur_best:
                best[nxt] = (ncost, nhops)
                prev[nxt] = (node, length, color, rtype)
                heapq.heappush(pq, (ncost, nhops, nxt))

    if end not in best:
        return None

    edges = []
    node = end
    while node != start:
        if node not in prev:
            return None
        prev_node, length, color, rtype = prev[node]
        edges.append({
            'a': prev_node,
            'b': node,
            'length': length,
            'color': color,
            'type': rtype,
        })
        node = prev_node
    edges.reverse()
    return edges


sections = split_mission_sections(missions_path.read_text())
standard = parse_missions(sections.get('Standard', ''))
highways = parse_missions(sections.get('Highways', ''))
all_missions = standard + highways


def missions_to_edges(missions):
    edges = []
    skipped = 0
    for a, b in missions:
        if a not in adj or b not in adj:
            skipped += 1
            continue
        path_edges = shortest_path_edges(a, b)
        if not path_edges:
            skipped += 1
            continue
        edges.extend(path_edges)
    return edges, skipped


all_edges, all_skipped = missions_to_edges(all_missions)
std_edges, std_skipped = missions_to_edges(standard)
hi_edges, hi_skipped = missions_to_edges(highways)

summarize_edges(all_edges, f"All missions stats (shortest paths) | missions: {len(all_missions)}, skipped: {all_skipped}")
summarize_edges(std_edges, f"Standard missions stats | missions: {len(standard)}, skipped: {std_skipped}")
summarize_edges(hi_edges, f"Highways missions stats | missions: {len(highways)}, skipped: {hi_skipped}")


# Write stats to stats.md
stats_md_path = Path('stats.md')
with stats_md_path.open('w') as f:
    def w(line=''):
        f.write(line + "\n")

    def summarize_edges_to_file(edges, label):
        total_len = sum(e['length'] for e in edges)
        total_edges = len(edges)
        land_len = sum(e['length'] for e in edges if e['type'] == 'land')
        ocean_len = sum(e['length'] for e in edges if e['type'] == 'ocean')
        land_edges = sum(1 for e in edges if e['type'] == 'land')
        ocean_edges = sum(1 for e in edges if e['type'] == 'ocean')
        color_totals = defaultdict(float)
        for e in edges:
            color_totals[e['color']] += e['length']

        w(label)
        w(f"  edges: {total_edges}, total length: {total_len:.2f}")
        if total_edges:
            w(f"  land edges: {land_edges} ({land_edges/total_edges:.1%})")
            w(f"  ocean edges: {ocean_edges} ({ocean_edges/total_edges:.1%})")
        if total_len > 0:
            w(f"  land length: {land_len:.2f} ({land_len/total_len:.1%})")
            w(f"  ocean length: {ocean_len:.2f} ({ocean_len/total_len:.1%})")
        w("  color totals (by length):")
        for color in sorted(color_totals):
            if total_len > 0:
                pct = color_totals[color] / total_len
                w(f"    {color}: {color_totals[color]:.2f} ({pct:.1%})")
            else:
                w(f"    {color}: {color_totals[color]:.2f}")
        w()

    summarize_edges_to_file(routes, "All routes stats")
    summarize_edges_to_file(all_edges, f"All missions stats (shortest paths) | missions: {len(all_missions)}, skipped: {all_skipped}")
    summarize_edges_to_file(std_edges, f"Standard missions stats | missions: {len(standard)}, skipped: {std_skipped}")
    summarize_edges_to_file(hi_edges, f"Highways missions stats | missions: {len(highways)}, skipped: {hi_skipped}")


All routes stats
  edges: 60, total length: 190.00
  land edges: 41 (68.3%)
  ocean edges: 19 (31.7%)
  land length: 92.00 (48.4%)
  ocean length: 98.00 (51.6%)
  color totals (by length):
    black: 28.00 (14.7%)
    green: 30.00 (15.8%)
    grey: 32.00 (16.8%)
    purple: 20.00 (10.5%)
    red: 23.00 (12.1%)
    white: 29.00 (15.3%)
    yellow: 28.00 (14.7%)

All missions stats (shortest paths) | missions: 100, skipped: 0
  edges: 336, total length: 945.00
  land edges: 222 (66.1%)
  ocean edges: 114 (33.9%)
  land length: 468.00 (49.5%)
  ocean length: 477.00 (50.5%)
  color totals (by length):
    black: 103.00 (10.9%)
    green: 213.00 (22.5%)
    grey: 174.00 (18.4%)
    purple: 141.00 (14.9%)
    red: 131.00 (13.9%)
    white: 111.00 (11.7%)
    yellow: 72.00 (7.6%)

Standard missions stats | missions: 80, skipped: 0
  edges: 250, total length: 693.00
  land edges: 165 (66.0%)
  ocean edges: 85 (34.0%)
  land length: 335.00 (48.3%)
  ocean length: 358.00 (51.7%)
  color totals (